<a href="https://colab.research.google.com/github/ADRopentech/International_Trade_Network_Analysis/blob/main/SNA_Calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from statsmodels.distributions.empirical_distribution import ECDF
import networkx.algorithms.community as nx_comm
import plotly.express as px
#import community as louvain
from sklearn.metrics import confusion_matrix

#Plotting parameters
sns.set(rc={"figure.dpi":300, "savefig.dpi":300})
sns.set_context('notebook')
sns.set_style("ticks")

In [2]:
#import file
dta = pd.read_csv('rice_trade_network.csv')
nodes_tab = pd.read_csv('nodes_table.csv')

nodes_tab['country_code'] = [
    '00' + str(x) if len(str(x)) == 1
    else '0' +  str(x) if len(str(x)) == 2
    else str(x)
    for x in nodes_tab['country_code'].tolist()
]

for col in ['exporter','importer']:
  dta[col] = [
      '00' + str(x) if len(str(x)) == 1
      else '0' +  str(x) if len(str(x)) == 2
      else str(x)
      for x in dta[col].tolist()
  ]

#create list of nodes
nodes = nodes_tab.country_code.values

#create list of (weighted) edges
edges = list(zip(dta.exporter.values,dta.importer.values, dta.final_trade_quantity.values))

In [3]:
#initialize the (weighted and directed) graph
G = nx.DiGraph()

#insert nodes
G.add_nodes_from(nodes)
#and their 'Zone' attribute
for i in range(len(nodes)):
    G.nodes[nodes[i]]['Zone'] = nodes_tab.Zone.values[i]

#insert weighted edges
G.add_weighted_edges_from(edges)

In [4]:
#Create list of nodes for each zone
zones = [z for z in nodes_tab.Zone.unique()]
d = {}
for z in zones:
    d[z] = nodes_tab.country_code[nodes_tab.Zone == z].values

In [58]:
ISO_map = pd.read_csv('ISO_Mapping.csv')

In [64]:
display(ISO_map)

,Unnamed: 0,index,Ccode,ISO
0,0,0,000,W00
1,1,1,180,COD
2,2,2,192,CUB
3,3,3,196,CYP
4,4,4,470,MLT
...,...,...,...,...
225,225,2279,064,BTN
226,226,2587,010,ATA
227,227,2615,654,SHN
228,228,3351,086,IOT


In [63]:
ISO_map['Ccode'] = [
    '00' + str(x) if len(str(x)) == 1
    else '0' +  str(x) if len(str(x)) == 2
    else str(x)
    for x in ISO_map['Ccode'].tolist()
]

In [65]:
iso_map = ISO_map.set_index('Ccode')['ISO'].to_dict()

In [66]:
print(iso_map)

{'000': 'W00', '180': 'COD', '192': 'CUB', '196': 'CYP', '470': 'MLT', '516': 'NAM', '591': 'PAN', '620': 'PRT', '699': 'IND', '724': 'ESP', '398': 'KAZ', '616': 'POL', '643': 'RUS', '804': 'UKR', '842': 'USA', '068': 'BOL', '076': 'BRA', '124': 'CAN', '152': 'CHL', '188': 'CRI', '376': 'ISR', '400': 'JOR', '484': 'MEX', '528': 'NLD', '682': 'SAU', '792': 'TUR', '826': 'GBR', '032': 'ARG', '156': 'CHN', '344': 'HKG', '554': 'NZL', '899': '_X ', '008': 'ALB', '056': 'BEL', '070': 'BIH', '100': 'BGR', '191': 'HRV', '203': 'CZE', '208': 'DNK', '233': 'EST', '246': 'FIN', '251': 'FRA', '270': 'GMB', '276': 'DEU', '300': 'GRC', '348': 'HUN', '372': 'IRL', '380': 'ITA', '428': 'LVA', '440': 'LTU', '442': 'LUX', '466': 'MLI', '579': 'NOR', '642': 'ROU', '686': 'SEN', '688': 'SRB', '703': 'SVK', '705': 'SVN', '710': 'ZAF', '752': 'SWE', '757': 'CHE', '807': 'MKD', '368': 'IRQ', '414': 'KWT', '430': 'LBR', '508': 'MOZ', '512': 'OMN', '562': 'NER', '586': 'PAK', '702': 'SGP', '784': 'ARE', '837'

# Density and Diameter Analysis

In [5]:
#Number of nodes and links
print('Number of nodes:', G.number_of_nodes())
print('Number of links:', G.number_of_edges())
print('Density:', nx.density(G))

Number of nodes: 231
Number of links: 4806
Density: 0.09045736871823828


In [82]:
print('Is there any isolated node?', bool(list(nx.isolates(G))))
print('\n')
print('Is the graph strongly connected?', nx.is_strongly_connected(G))
print('Number of strongly connected components:', nx.number_strongly_connected_components(G))
print('Strongly Connected Node: ', len(list(nx.strongly_connected_components(G))))

Is there any isolated node? False


Is the graph strongly connected? False
Number of strongly connected components: 51
Strongly Connected Node:  51


In [7]:
comp_size = pd.Series([len(c) for c in nx.strongly_connected_components(G)])

comp_size.value_counts()
#Comp size | Frequency

1      50
181     1
Name: count, dtype: int64

In [83]:
print('Is the graph weakly connected?', nx.is_weakly_connected(G))
print('Number of weakly connected components:', nx.number_weakly_connected_components(G))
print('Weakly Connected Node: ', len(list(nx.weakly_connected_components(G))))

Is the graph weakly connected? True
Number of weakly connected components: 1
Weakly Connected Node:  1


In [84]:
print('Is the graph semi connected?', nx.is_semiconnected(G))

Is the graph semi connected? False


In [9]:
#heaviest edges
#weight is to be understood as thousand Kgs
dta.sort_values('final_trade_quantity', ascending=False).head()

,Unnamed: 0,exporter,importer,final_trade_quantity,exporter_country,importer_country,SourceZone,TargetZone
3553,2963,704,608,3.215154e+09,Viet Nam,Philippines,Asia,Asia
3256,1007,699,156,2.180495e+09,India,China,Asia,Asia
3266,1252,699,204,1.630420e+09,India,Benin,Asia,Africa
3926,1990,764,368,1.600163e+09,Thailand,Iraq,Asia,Asia
3358,3102,699,686,1.355147e+09,India,Senegal,Asia,Africa


In [10]:
#does density change in different zones?
dens = {}
for z in zones:
    print('Density in ', z, ' (', len(d[z]), ' countries)' ': ', nx.density(G.subgraph(d[z])), sep='')
    dens[z] = nx.density(G.subgraph(d[z]))

Density in Asia (51 countries): 0.25019607843137254
Density in Europe (44 countries): 0.43023255813953487
Density in Antarctica (1 countries): 0
Density in Africa (56 countries): 0.059415584415584415
Density in Oceania (25 countries): 0.058333333333333334
Density in Americas (50 countries): 0.11755102040816326
Density in Special categories and unspecified areas (4 countries): 0


In [43]:
dens['Network'] = nx.density(G)
dens = pd.Series(dens)
'''
#Plot
fig, ax = plt.subplots(figsize=(12,8), dpi=300)
sns.barplot(x=dens.index, y=dens.values, ax=ax, palette='Set2',
            order=dens.sort_values(ascending=False).index)
ax.bar_label(ax.containers[0], fmt='%.2f')
sns.despine()
#plt.show()'''

"\n#Plot\nfig, ax = plt.subplots(figsize=(12,8), dpi=300)\nsns.barplot(x=dens.index, y=dens.values, ax=ax, palette='Set2', \n            order=dens.sort_values(ascending=False).index)\nax.bar_label(ax.containers[0], fmt='%.2f')\nsns.despine()\n#plt.show()"

In [12]:
#Africa: nr of in/out going links
africa_in = dta.SourceZone[dta.TargetZone=='Africa'].value_counts(ascending=False).reset_index()
africa_out = dta.TargetZone[dta.SourceZone=='Africa'].value_counts(ascending=False).reset_index()
w_africa_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Africa'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_africa_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Africa'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Asia: nr of in/out going links
asia_in = dta.SourceZone[dta.TargetZone=='Asia'].value_counts(ascending=False).reset_index()
asia_out = dta.TargetZone[dta.SourceZone=='Asia'].value_counts(ascending=False).reset_index()
w_asia_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Asia'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_asia_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Asia'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Europe: nr of in/out going links
europe_in = dta.SourceZone[dta.TargetZone=='Europe'].value_counts(ascending=False).reset_index()
europe_out = dta.TargetZone[dta.SourceZone=='Europe'].value_counts(ascending=False).reset_index()
w_europe_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Europe'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_europe_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Europe'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Americas: nr of in/out going links
na_in = dta.SourceZone[dta.TargetZone=='Americas'].value_counts(ascending=False).reset_index()
na_out = dta.TargetZone[dta.SourceZone=='Americas'].value_counts(ascending=False).reset_index()
w_na_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Americas'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_na_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Americas'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Oceania: nr of in/out going links
oceania_in = dta.SourceZone[dta.TargetZone=='Oceania'].value_counts(ascending=False).reset_index()
oceania_out = dta.TargetZone[dta.SourceZone=='Oceania'].value_counts(ascending=False).reset_index()
w_oceania_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Oceania'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_oceania_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Oceania'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Antarctica: nr of in/out going links
sa_in = dta.SourceZone[dta.TargetZone=='Antarctica'].value_counts(ascending=False).reset_index()
sa_out = dta.TargetZone[dta.SourceZone=='Antarctica'].value_counts(ascending=False).reset_index()
w_sa_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Antarctica'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_sa_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Antarctica'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Special Categories: nr of in/out going links
scua_in = dta.SourceZone[dta.TargetZone=='Special categories and unspecified areas'].value_counts(ascending=False).reset_index()
scua_out = dta.TargetZone[dta.SourceZone=='Special categories and unspecified areas'].value_counts(ascending=False).reset_index()
w_scua_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Special categories and unspecified areas'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_scua_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Special categories and unspecified areas'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

In [13]:
#Africa: nr of in/out going links
africa_in = dta.SourceZone[dta.TargetZone=='Africa'].value_counts(ascending=False).reset_index()
africa_out = dta.TargetZone[dta.SourceZone=='Africa'].value_counts(ascending=False).reset_index()
w_africa_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Africa'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_africa_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Africa'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Asia: nr of in/out going links
asia_in = dta.SourceZone[dta.TargetZone=='Asia'].value_counts(ascending=False).reset_index()
asia_out = dta.TargetZone[dta.SourceZone=='Asia'].value_counts(ascending=False).reset_index()
w_asia_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Asia'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_asia_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Asia'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Europe: nr of in/out going links
europe_in = dta.SourceZone[dta.TargetZone=='Europe'].value_counts(ascending=False).reset_index()
europe_out = dta.TargetZone[dta.SourceZone=='Europe'].value_counts(ascending=False).reset_index()
w_europe_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Europe'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_europe_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Europe'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Americas: nr of in/out going links
na_in = dta.SourceZone[dta.TargetZone=='Americas'].value_counts(ascending=False).reset_index()
na_out = dta.TargetZone[dta.SourceZone=='Americas'].value_counts(ascending=False).reset_index()
w_na_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Americas'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_na_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Americas'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Oceania: nr of in/out going links
oceania_in = dta.SourceZone[dta.TargetZone=='Oceania'].value_counts(ascending=False).reset_index()
oceania_out = dta.TargetZone[dta.SourceZone=='Oceania'].value_counts(ascending=False).reset_index()
w_oceania_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Oceania'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_oceania_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Oceania'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Antarctica: nr of in/out going links
sa_in = dta.SourceZone[dta.TargetZone=='Antarctica'].value_counts(ascending=False).reset_index()
sa_out = dta.TargetZone[dta.SourceZone=='Antarctica'].value_counts(ascending=False).reset_index()
w_sa_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Antarctica'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_sa_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Antarctica'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

#Special Categories: nr of in/out going links
scua_in = dta.SourceZone[dta.TargetZone=='Special categories and unspecified areas'].value_counts(ascending=False).reset_index()
scua_out = dta.TargetZone[dta.SourceZone=='Special categories and unspecified areas'].value_counts(ascending=False).reset_index()
w_scua_in = dta[['SourceZone', 'final_trade_quantity']][dta.TargetZone=='Special categories and unspecified areas'].groupby('SourceZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()
w_scua_out = dta[['TargetZone', 'final_trade_quantity']][dta.SourceZone=='Special categories and unspecified areas'].groupby('TargetZone').sum().sort_values('final_trade_quantity', ascending=False).reset_index()

In [14]:
#indegree
asia_in['TargetZone'] = 'Asia'
africa_in['TargetZone'] = 'Africa'
sa_in['TargetZone'] = 'Antarctica'
oceania_in['TargetZone'] = 'Oceania'
europe_in['TargetZone'] = 'Europe'
na_in['TargetZone'] = 'Americas'
scua_in['TargetZone'] = 'Special categories and unspecified areas'

#outdegree
asia_out['SourceZone'] = 'Asia'
africa_out['SourceZone'] = 'Africa'
sa_out['SourceZone'] = 'Antarctica'
oceania_out['SourceZone'] = 'Oceania'
europe_out['SourceZone'] = 'Europe'
na_out['SourceZone'] = 'Americas'
scua_out['SourceZone'] = 'Special categories and unspecified areas'

#weighted indegree
w_asia_in['TargetZone'] = 'Asia'
w_africa_in['TargetZone'] = 'Africa'
w_sa_in['TargetZone'] = 'Antarctica'
w_oceania_in['TargetZone'] = 'Oceania'
w_europe_in['TargetZone'] = 'Europe'
w_na_in['TargetZone'] = 'Americas'
w_scua_in['TargetZone'] = 'Special categories and unspecified areas'

#weighted outdegree
w_asia_out['SourceZone'] = 'Asia'
w_africa_out['SourceZone'] = 'Africa'
w_sa_out['SourceZone'] = 'Antarctica'
w_oceania_out['SourceZone'] = 'Oceania'
w_europe_out['SourceZone'] = 'Europe'
w_na_out['SourceZone'] = 'Americas'
w_scua_out['SourceZone'] = 'Special categories and unspecified areas'

In [15]:
#Concatenate
zone_indegree = pd.concat([asia_in, africa_in, sa_in, oceania_in, europe_in, na_in, scua_in])
zone_indegree.rename(columns={ 'count' : 'Nr of connections'}, inplace=True)

zone_outdegree = pd.concat([asia_out, africa_out, sa_out, oceania_out, europe_out, na_out, scua_out])
zone_outdegree.rename(columns={ 'count' : 'Nr of connections'}, inplace=True)

zone_w_indegree = pd.concat([w_asia_in, w_africa_in, w_sa_in, w_oceania_in, w_europe_in, w_na_in, w_scua_in])

zone_w_outdegree = pd.concat([w_asia_out, w_africa_out, w_sa_out, w_oceania_out, w_europe_out, w_na_out, w_scua_out])

In [16]:
#How much each zone exports
zone_w_indegree[zone_w_indegree['SourceZone'] != zone_w_indegree['TargetZone']].groupby('SourceZone').sum().reset_index().sort_values(by='final_trade_quantity', ascending=False)

,SourceZone,final_trade_quantity,TargetZone
2,Asia,2.584866e+10,AfricaOceaniaEuropeAmericasSpecial categories ...
1,Americas,2.370862e+09,AsiaAfricaOceaniaEuropeSpecial categories and ...
4,Oceania,3.733468e+08,AsiaAfricaEuropeAmericasSpecial categories and...
3,Europe,2.333296e+08,AsiaAfricaAntarcticaOceaniaAmericasSpecial cat...
0,Africa,9.478241e+06,AsiaAntarcticaOceaniaEuropeAmericasSpecial cat...
5,Special categories and unspecified areas,3.695768e+06,AsiaAfricaEuropeAmericas


In [17]:
#How much each zone imports
zone_w_outdegree[zone_w_outdegree['SourceZone'] != zone_w_outdegree['TargetZone']].groupby('TargetZone').sum().reset_index().sort_values(by='final_trade_quantity', ascending=False)

,TargetZone,final_trade_quantity,SourceZone
0,Africa,2.094815e+10,AsiaOceaniaEuropeAmericasSpecial categories an...
4,Europe,4.218653e+09,AsiaAfricaOceaniaAmericasSpecial categories an...
1,Americas,1.641243e+09,AsiaAfricaOceaniaEuropeSpecial categories and ...
3,Asia,1.096838e+09,AfricaOceaniaEuropeAmericasSpecial categories ...
5,Oceania,7.160562e+08,AsiaAfricaEuropeAmericas
6,Special categories and unspecified areas,2.184114e+08,AsiaAfricaOceaniaEuropeAmericas
2,Antarctica,1.418380e+04,AfricaEurope


In [18]:
#Most important edges
zone_w_indegree.sort_values('final_trade_quantity', ascending = False).head()

,SourceZone,final_trade_quantity,TargetZone
0,Asia,2.684039e+10,Asia
0,Asia,2.041072e+10,Africa
0,Americas,5.446218e+09,Americas
0,Asia,3.161531e+09,Europe
1,Europe,2.481669e+09,Europe


In [42]:
'''#From where each zone (horiz axis) imports rice (in terms of number of connections)
fig, ax = plt.subplots(figsize=(10,8))
sns.barplot(x='TargetZone', y='Nr of connections', hue='SourceZone', data=zone_indegree, palette='Set2', ax=ax)
ax.tick_params(axis='x', labelsize=10)
ax.set_xlabel('')
ax.set_ylabel('Number of connections')
#plt.show()'''

"#From where each zone (horiz axis) imports rice (in terms of number of connections)\nfig, ax = plt.subplots(figsize=(10,8))\nsns.barplot(x='TargetZone', y='Nr of connections', hue='SourceZone', data=zone_indegree, palette='Set2', ax=ax)\nax.tick_params(axis='x', labelsize=10)\nax.set_xlabel('')\nax.set_ylabel('Number of connections')\n#plt.show()"

In [41]:
'''#To which zone each zone (horiz axis) exports rice (in terms of connections)
fig, ax = plt.subplots(figsize=(10,8))
sns.barplot(x='SourceZone', y='Nr of connections', hue='TargetZone', data=zone_outdegree, palette='Set2', ax=ax)
ax.tick_params(axis='x', labelsize=8)
ax.set_xlabel('')
ax.set_ylabel('Number of connections')
plt.legend(loc = 'upper right')
#plt.show()'''

"#To which zone each zone (horiz axis) exports rice (in terms of connections)\nfig, ax = plt.subplots(figsize=(10,8))\nsns.barplot(x='SourceZone', y='Nr of connections', hue='TargetZone', data=zone_outdegree, palette='Set2', ax=ax)\nax.tick_params(axis='x', labelsize=8)\nax.set_xlabel('')\nax.set_ylabel('Number of connections')\nplt.legend(loc = 'upper right')\n#plt.show()"

In [38]:
'''##From where each zone (horiz axis) imports rice (in terms of number of quantity)
fig, ax = plt.subplots(figsize=(10,8))
sns.barplot(x='TargetZone', y='final_trade_quantity', hue='SourceZone', data=zone_w_indegree, palette='Set2', ax=ax)
ax.tick_params(axis='x', labelsize=10)
ax.set_ylabel('Trades value (kgs)')
#plt.show()'''

"##From where each zone (horiz axis) imports rice (in terms of number of quantity)\nfig, ax = plt.subplots(figsize=(10,8))\nsns.barplot(x='TargetZone', y='final_trade_quantity', hue='SourceZone', data=zone_w_indegree, palette='Set2', ax=ax)\nax.tick_params(axis='x', labelsize=10)\nax.set_ylabel('Trades value (kgs)')\n#plt.show()"

In [39]:
'''#To which zone each zone (horiz axis) exports rice (in terms of connections)
fig, ax = plt.subplots(figsize=(12,8))
sns.barplot(x='SourceZone', y='final_trade_quantity', hue='TargetZone', data=zone_w_outdegree, palette='Set2', ax=ax)
ax.tick_params(axis='x', labelsize=10)
sns.despine()
ax.set_ylabel('Trades value (kilo)')
plt.legend(title='TargetZone', loc = 'upper right')
#plt.show()'''

"#To which zone each zone (horiz axis) exports rice (in terms of connections)\nfig, ax = plt.subplots(figsize=(12,8))\nsns.barplot(x='SourceZone', y='final_trade_quantity', hue='TargetZone', data=zone_w_outdegree, palette='Set2', ax=ax)\nax.tick_params(axis='x', labelsize=10)\nsns.despine()\nax.set_ylabel('Trades value (kilo)')\nplt.legend(title='TargetZone', loc = 'upper right')\n#plt.show()"

# Centrality Analysis

In [23]:
# @title out-degree
out_degree = pd.Series(dict(G.out_degree))
out_degree_values = list(out_degree.values)

print('Mean out-degree:', np.mean(out_degree_values))
print('Median out-degree:', np.median(out_degree_values))
print('Out-degree std dev:', np.std(out_degree_values))
print('Max out-degree:', np.max(out_degree_values))
print('Min out-degree:', np.min(out_degree_values))

Mean out-degree: 20.805194805194805
Median out-degree: 6.0
Out-degree std dev: 34.10349880910545
Max out-degree: 176
Min out-degree: 0


In [24]:
# @title out-degree
in_degree = pd.Series(dict(G.in_degree))
in_degree_values = list(in_degree.values)

print('Mean in-degree:', np.mean(in_degree_values))
print('Median in-degree:', np.median(in_degree_values))
print('In-degree std dev:', np.std(in_degree_values))
print('Max in-degree:', np.max(in_degree_values))
print('Min in-degree:', np.min(in_degree_values))

Mean in-degree: 20.805194805194805
Median in-degree: 16.0
In-degree std dev: 17.45398821518253
Max in-degree: 83
Min in-degree: 0


In [25]:
# @title total degree
degree = pd.Series(dict(G.degree))
degree_values = list(degree.values)

print('Mean degree:', np.mean(degree_values))
print('Median degree:', np.median(degree_values))
print('Degree std dev:', np.std(degree_values))
print('Max degree:', np.max(degree_values))
print('Min degree:', np.min(degree_values))

Mean degree: 41.61038961038961
Median degree: 24.0
Degree std dev: 46.6964469789112
Max degree: 248
Min degree: 1


In [26]:
#higher in-degree, out-degree, total-degree
print('Countries with higher import (in-degree):')
print(in_degree.sort_values(ascending=False).head())
print('\n')
print('Countries with higher export (out-degree):')
print(out_degree.sort_values(ascending=False).head())
print('\n')
print('Countries with larger number of connections (degree):')
print(degree.sort_values(ascending=False).head())

Countries with higher import (in-degree):
528    83
124    82
842    81
251    74
276    72
dtype: int64


Countries with higher export (out-degree):
699    176
842    167
764    163
380    144
156    143
dtype: int64


Countries with larger number of connections (degree):
842    248
699    214
764    200
380    198
792    192
dtype: int64


In [27]:
# @title degree centrality
degree_centrality = nx.degree_centrality(G)
print('Degree Centrality of graph:\n')
print(degree_centrality)
print(len(degree_centrality))
print('\n')
in_degree_centrality = nx.in_degree_centrality(G)
print('In-Degree Centrality of graph:\n')
print(in_degree_centrality)
print(len(in_degree_centrality))
print('\n')
out_degree_centrality = nx.out_degree_centrality(G)
print('Out-Degree Centrality of graph:\n')
print(out_degree_centrality)
print(len(out_degree_centrality))

Degree Centrality of graph:

{'004': 0.043478260869565216, '008': 0.08695652173913043, '010': 0.008695652173913044, '012': 0.052173913043478265, '016': 0.021739130434782608, '020': 0.0391304347826087, '024': 0.15217391304347827, '028': 0.09565217391304348, '031': 0.13478260869565217, '032': 0.2565217391304348, '036': 0.4043478260869565, '040': 0.4217391304347826, '044': 0.11739130434782609, '048': 0.26956521739130435, '050': 0.21739130434782608, '051': 0.09565217391304348, '052': 0.0782608695652174, '056': 0.6652173913043479, '060': 0.043478260869565216, '064': 0.021739130434782608, '068': 0.09565217391304348, '070': 0.11739130434782609, '072': 0.07391304347826087, '076': 0.5521739130434783, '084': 0.052173913043478265, '086': 0.004347826086956522, '090': 0.008695652173913044, '092': 0.034782608695652174, '096': 0.10434782608695653, '100': 0.3739130434782609, '104': 0.3173913043478261, '108': 0.0782608695652174, '112': 0.030434782608695653, '116': 0.3782608695652174, '120': 0.086956521

In [28]:
# @title closeness centrality
closeness_centrality = nx.closeness_centrality(G)
print('Closeness Centrality of graph:\n')
print(closeness_centrality)
print(len(closeness_centrality))

Closeness Centrality of graph:

{'004': 0.3158276125095347, '008': 0.3636802810715854, '010': 0.307831602169317, '012': 0.3280578389620679, '016': 0.29415019762845845, '020': 0.3310744627686157, '024': 0.39029103334511606, '028': 0.3776760097108806, '031': 0.37504528985507246, '032': 0.35126193001060446, '036': 0.41865520728008093, '040': 0.4223383909218411, '044': 0.3881870385561936, '048': 0.4162352349836642, '050': 0.3243634939287113, '051': 0.37117884356790676, '052': 0.3683309240520405, '056': 0.460119461036255, '060': 0.35736325385694245, '064': 0.27121919266355526, '068': 0.3341470795924544, '070': 0.37022465630937745, '072': 0.35212076113532476, '076': 0.35212076113532476, '084': 0.346195652173913, '086': 0.302083709182753, '090': 0.27681434947925276, '092': 0.33648923201950426, '096': 0.3529838022165388, '100': 0.3945681953543776, '104': 0.3137633797480345, '108': 0.37799840237361637, '112': 0.3097148200093502, '116': 0.3437169243540521, '120': 0.36927536231884056, '124': 0.49

In [29]:
# @title betweenness centrality
betweenness_centrality = nx.betweenness_centrality(G)
print('betweenness Centrality of graph:\n')
print(betweenness_centrality)
print(len(betweenness_centrality))

betweenness Centrality of graph:

{'004': 2.5180882025385542e-05, '008': 4.488893732146653e-05, '010': 0.0, '012': 6.546944868177265e-07, '016': 0.0, '020': 1.40332291220245e-05, '024': 0.005604964716311732, '028': 0.0, '031': 0.0004234112207024667, '032': 0.00038481804551487327, '036': 0.011598158292800215, '040': 0.006940788082078035, '044': 0.004459661186267964, '048': 0.002761738207039685, '050': 0.00011247017437795606, '051': 0.00038941176583354143, '052': 0.0002713450286700709, '056': 0.0269077508752763, '060': 0.0, '064': 0.0, '068': 4.562878593129226e-05, '070': 0.00011616653853068421, '072': 0.001979864178431878, '076': 0.005701517135342231, '084': 4.7762690932344204e-05, '086': 0.0, '090': 0.0, '092': 0.00011965278700487368, '096': 6.683421625793394e-05, '100': 0.0024921202340773786, '104': 0.00034103825676124724, '108': 9.468525582197386e-06, '112': 3.1144373360762297e-06, '116': 0.0013963310791901995, '120': 0.0001890803274554301, '124': 0.04795380815230758, '132': 3.640256

#K-Core

In [30]:
k_core = nx.k_core(G)
print(k_core)

DiGraph with 42 nodes and 1181 edges


# Modularity - Community Detection

In [31]:
#Greedy approach
greedy_comm = list(nx_comm.greedy_modularity_communities(G, weight='final_trade_quantity'))
print('Number of communities detected:', len(greedy_comm))

Number of communities detected: 4


In [32]:
print(greedy_comm[3])
print(len(greedy_comm[3]))

frozenset({'678', '788', '148', '270', '336', '466', '686', '226', '666', '251', '020', '540', '418', '694', '132'})
15


In [51]:
dfs = [[] for i in range(len(greedy_comm))]
for i in range(len(greedy_comm)):
    dfs[i] = pd.DataFrame({'Country' : list(greedy_comm[i]), 'Greedy Comm.' : ('Comm. ' + str(i+1))})

greedy_df = pd.concat(dfs)

greedy_df['country'] = greedy_df.Country.map(country_codes)

#print(greedy_df)
#print(nodes_tab)

In [55]:
greedy_df.rename(columns={'Country':'CCode'}, inplace=True)

In [68]:
greedy_df['ISO'] = greedy_df.CCode.map(iso_map)

In [72]:
display(greedy_df[greedy_df['ISO']=='NaN'])

,CCode,Greedy Comm.,country,ISO


In [73]:
fig = px.choropleth(greedy_df, locations="ISO", color="Greedy Comm.", color_discrete_sequence=px.colors.qualitative.T10)
#fig.show()

In [40]:
'''fig, ax = plt.subplots(figsize=(12,8), dpi=300)
sns.countplot(x='Greedy Comm.', data=greedy_df, ax=ax)
ax.set_xlabel('')
ax.bar_label(ax.containers[0])
#plt.show()'''

"fig, ax = plt.subplots(figsize=(12,8), dpi=300)\nsns.countplot(x='Greedy Comm.', data=greedy_df, ax=ax)\nax.set_xlabel('')\nax.bar_label(ax.containers[0])\n#plt.show()"

In [35]:
#In-degree
w_in_degree[list(greedy_comm[3])].sort_values(ascending=False).head()

NameError: name 'w_in_degree' is not defined

In [ ]:
#Out-degree
w_out_degree[list(greedy_comm[3])].sort_values(ascending=False).head()

#Small World Analysis
##Clustering Coefficient
##Average clustering Coefficient
##Average Path length

In [74]:
# @title clustering co-efficient

clustering_coeff = nx.clustering(G, weight = 'final_trade_quantity')
print(clustering_coeff)

{'004': 0.5555555555555556, '008': 0.6931216931216931, '010': 1.0, '012': 0.7803030303030303, '016': 0.4, '020': 0.7428571428571429, '024': 0.4746192893401015, '028': 0.45670995670995673, '031': 0.5672451193058569, '032': 0.5129259694477086, '036': 0.4179981203007519, '040': 0.441265385445908, '044': 0.4328571428571429, '048': 0.45986177565124936, '050': 0.5594262295081968, '051': 0.665938864628821, '052': 0.5723684210526315, '056': 0.2966781708369284, '060': 0.8863636363636364, '064': 0.7, '068': 0.6347826086956522, '070': 0.670487106017192, '072': 0.6240601503759399, '076': 0.24367643375907838, '084': 0.5909090909090909, '086': 0, '090': 0.5, '092': 0.3392857142857143, '096': 0.6985294117647058, '100': 0.493942731277533, '104': 0.3953488372093023, '108': 0.7581699346405228, '112': 0.5952380952380952, '116': 0.3665952317171176, '120': 0.6904761904761905, '124': 0.2590408421873477, '132': 0.7433862433862434, '136': 0.6666666666666666, '140': 0.7575757575757576, '144': 0.452526799387442

In [75]:
# @title average clustering

avg_cluster = nx.average_clustering(G, weight = 'final_trade_quantity')

In [76]:
print(avg_cluster)

0.536968021445557


In [85]:
# @title avg path length
#avg_path_length = nx.average_shortest_path_length(G, weight = 'final_trade_quantity') # not well connected graph

#Bonacich Power

In [86]:
# @title Adjacency Matrix of Graph as a numpy array

A = nx.to_numpy_array(G, weight= 'final_trade_quantity')

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [87]:
#@title Transpose
A_T = A.T

In [88]:
A_sym = A + A_T

In [91]:
col_sums = A_sym.sum(axis=0)
col_sums[col_sums == 0] = 1
A_norm = A_sym/col_sums
print(A_norm)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [92]:
def normalise_cols(matrix):
  matrix_t = matrix.T
  matrix_sum = matrix+matrix_t
  col_sum = matrix_sum.sum(axis=0)
  matrix_norm = matrix_sum/col_sum

  return matrix_norm

In [93]:
def bonacich_power_centrality(graph, beta, alpha=None):
  A = nx.to_numpy_array(graph, weight='final_trade_quantity')

  A = normalise_cols(A)

  eigenvalues, _ = np.linalg.eig(A)
  lambda_max = max(eigenvalues)

  if alpha is None:
    alpha = np.sqrt(len(graph.nodes))

  I = np.eye(len(A))
  l = np.ones(len(A))

  centrality_vector = alpha * np.linalg.inv(I - beta * A) @ A @ l

  return  centrality_vector

In [94]:
beta = -0.5

In [95]:
bonachich_power = bonacich_power_centrality(G, beta)

In [97]:
for node, value in zip(G.nodes, bonachich_power):
    print(f"Node {node}: Bonacich power centrality = {value}")

Node 004: Bonacich power centrality = -0.22040411617497718
Node 008: Bonacich power centrality = 0.36591001121081596
Node 010: Bonacich power centrality = -0.27955046767167313
Node 012: Bonacich power centrality = -0.9078099319606929
Node 016: Bonacich power centrality = -0.6021876668895677
Node 020: Bonacich power centrality = -0.22538493851966962
Node 024: Bonacich power centrality = 4.128430019910469
Node 028: Bonacich power centrality = 7.710431681052767
Node 031: Bonacich power centrality = 2.151677753745242
Node 032: Bonacich power centrality = 4.76969987559925
Node 036: Bonacich power centrality = 11.933523268256238
Node 040: Bonacich power centrality = 16.70671261330529
Node 044: Bonacich power centrality = 5.946785689348662
Node 048: Bonacich power centrality = 9.258204173067256
Node 050: Bonacich power centrality = 1.6697698684944906
Node 051: Bonacich power centrality = 0.45764668239625705
Node 052: Bonacich power centrality = 2.8975917215566787
Node 056: Bonacich power cent